In [7]:
import os
import numpy as np
import csv
import xml.etree.ElementTree as ET

In [16]:
def sbr2xr(actuator):
    """
    Format: "Joint in Pepper": [pepper joint index, scale(-1: change direction)]
    """
    dic = {        
        "HipPitch": [2,1],
        "HipRoll": [3,1],
        "KneePitch": [0,1],  
        "HeadPitch": [5,1],
        "HeadYaw": [4,1],
 
        "LShoulderPitch": [7,-1], 
        "LShoulderRoll": [8,1.1], 
        "LElbowRoll": [10,1], 
        "LElbowYaw": [9,-1],
        "LWristYaw": [11,1],
        
        "RShoulderPitch": [14,-1], 
        "RShoulderRoll": [15,1.1], 
        "RElbowRoll": [17,-1], 
        "RElbowYaw": [16,-1],
        "RWristYaw": [18,1],
        
        "LHand": [21,22,23,24,25],
        "RHand": [26,27,28,29,30],   
    }
    if actuator in dic:
        return dic[actuator][0],dic[actuator][-1] 
    print actuator
    raise KeyError("No correponding conversion key found!")

In [17]:
def write2file(directory, res):
    """
    Write to file with .cvs format 
    """
    with open(directory,"w") as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerows(res)   

In [18]:
def linear_interpolation(res): 
    """
    Interpolate the key frame, write to _interp.csv file 
    """
    tmp = []
    res_interp = [[0 for col in range(len(res[0]))] for row in range(len(res))]
#     print ("==== The res_interp is {} X {} array ====".format(len(res_interp[0]), len(res_interp)) )
    for row in range(len(res[0])):
        x = []
        y = []
        cur_ptr = 0
        next_ptr = 0

        for index, i in enumerate(res):
            x.append(index)
            y.append(i[row])

        for index, value in enumerate(y):
            if value == 0:
                next_ptr = next_ptr + 1
                continue
            else: 
                scale = (y[next_ptr] - y[cur_ptr]) / (next_ptr * 1.0 - cur_ptr)          
                for i in range(cur_ptr, next_ptr):
                    y[i] = scale * (i-cur_ptr) + y[cur_ptr]

                cur_ptr = next_ptr
                next_ptr = next_ptr + 1  
        tmp.append(y)
        
    ############## Transpose ###################
    
    for rows in range(len(tmp)):
        for index, i in enumerate(tmp[rows]):
            res_interp[index][rows] = i
            
    return res_interp

In [25]:
def conversion(directory, w, prefix):
    tree = ET.parse(directory)
    root = tree.getroot()
    res = None
    
    ### check whether this file has action or not ###
    for neighbor in root.iter('Timeline'):
        if neighbor.attrib['enable'] == "0":
            continue
        framelen = neighbor.attrib['size']
        res = [[0 for col in range(DOF)] for row in range(int(framelen))]
        print ("==== The result is {} X {} array ====".format(DOF, int(framelen)) )
        
        if len(neighbor) is 0:
            raise KeyError("ActuatorList not found, check box.xar file")
        
        ### find the ActuatorList tag ###
        for actuator_list in neighbor:         
            if actuator_list.tag != "ActuatorList":
                continue
            for actuator_curve in actuator_list:
                if len(actuator_curve) is 0:
                    continue
                  
                joint_name = actuator_curve.attrib['actuator']
                va, ori = sbr2xr(joint_name)
                
                for key in actuator_curve:
                    index = int(key.attrib['frame'])-1
                    
                    ### radian 2 degree; change degree to align with ginger in UE4 ###
                    if joint_name == "LHand" or joint_name == "RHand":
                        res[index][va:va + 5] = [float(key.attrib['value'])]*5
                    elif joint_name == "LShoulderPitch" or joint_name == "RShoulderPitch":
                        res[index][va] = (float(key.attrib['value'])*ori + 90)*(np.pi/180) 
                    elif joint_name == "LElbowYaw":
                        res[index][va] = (float(key.attrib['value'])*ori + 90)*(np.pi/180) 
                        if res[index][va] > np.pi/2:
                            res[index][va] = res[index][va] - np.pi
                    elif joint_name == "RElbowYaw":
                        res[index][va] = (float(key.attrib['value'])*ori - 90)*(np.pi/180) 
                        if res[index][va] < -np.pi/2:
                            res[index][va] = res[index][va] + np.pi
                        
                    else:
                        res[index][va] = float(key.attrib['value'])*ori *(np.pi/180) 
#                         res[index][va] = ( float(key.attrib['value'])*np.pi*ori )/180  
    if w and (res is not None):
        write2file(directory.replace("box.xar", prefix + "box.csv"), res)
        write2file(directory.replace("box.xar", prefix + "box_interp.csv"), linear_interpolation(res))
        
    return res

In [26]:
DOF = 35
rootdir = os.getcwd()
file_num = 0
print "The Animation directory is: " + rootdir

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if ".xar" in file:
            tmp_dir = os.path.join(subdir, file)
            if "/Pepper/" in tmp_dir:
                file_num = file_num + 1
                file_prefix = tmp_dir.split("/");
                prefix = file_prefix[-5] + "_" + file_prefix[-4] + "_"  + file_prefix[-2] + "_"
                print prefix
                res = conversion(tmp_dir, True, prefix)

print("Animation have {} actions".format(file_num))
# tmp_dir = "/Users/jialiang/Desktop/extraction/Animation/Dialog animations/Question/Pepper/LittleArmsBumpInFront_LeanRight_01/box.xar"


The Animation directory is: /Users/jialiang/Desktop/Conversion
Moods_Positive_Happy_
==== The result is 35 X 87 array ====
Moods_Positive_Kisses_
==== The result is 35 X 127 array ====
Moods_Positive_Excited_
==== The result is 35 X 56 array ====
Moods_Negative_Fear_
==== The result is 35 X 107 array ====
Moods_Negative_Bored_
==== The result is 35 X 92 array ====
Moods_Negative_Confused_
==== The result is 35 X 82 array ====
Moods_Neutral_Curious_
==== The result is 35 X 67 array ====
Moods_Neutral_Thinking_
==== The result is 35 X 140 array ====
Moods_Neutral_Chill_
==== The result is 35 X 168 array ====
Entertainment_Music_Guitar_
==== The result is 35 X 287 array ====
Entertainment_Music_Saxophone_
==== The result is 35 X 262 array ====
Entertainment_Fun_Mystical_
==== The result is 35 X 199 array ====
Entertainment_Fun_TakePicture_
==== The result is 35 X 90 array ====
Entertainment_Fun_Vacuum_
==== The result is 35 X 256 array ====
Entertainment_Sports_Golf_
==== The result is 35

==== The result is 35 X 35 array ====
Dialog animations_Self & others_WidePointAtUserRightArm_LeanRight_01_
==== The result is 35 X 31 array ====
Dialog animations_Self & others_GoToStance_Self&Others_LeanFront_
==== The result is 35 X 31 array ====
Animation have 99 actions
